In [ ]:
# Создание датафрейма из исходной таблицы
import pandas as pd
from nebula3.gclient.net import ConnectionPool
from nebula3.Config import Config
import time

df = pd.read_csv('data_test.csv', sep = ';', header = 0)
print(df.info(),'\n','\n', df.head(),'\n','\n', df.tail())

In [ ]:
# Проверка количества уникальных значений по столбцам
for i in range(df.shape[1]):
    print(df.columns[i],'=', len(df.iloc[:, i].drop_duplicates()))

In [ ]:
# Стандартное подключение к БД
config = Config()
config.max_connection_pool_size = 10

connection_pool = ConnectionPool()

ok = connection_pool.init([('127.0.0.1', 9669)], config)
session = connection_pool.get_session('root', 'nebula')

In [ ]:
print(df.columns[1], df.columns[2], df.columns[0])

In [ ]:
# Формирование пространства БД, тегов и ребер 
session.execute('CREATE SPACE IF NOT EXISTS data_1503 ('
        'PARTITION_NUM=15,'
        'vid_type=FIXED_STRING(100));'
        'USE data_1503;'
        'CREATE TAG IF NOT EXISTS `{}`(name string);'
        'CREATE TAG IF NOT EXISTS `{}`(name string);'
        'CREATE EDGE IF NOT EXISTS `{}`(id int)'.format(df.columns[1], df.columns[2], df.columns[0])
        )

In [ ]:
# Проверка созданного пространства БД, наличия тегов и ребер
print(session.execute('SHOW HOSTS'))
print(session.execute('SHOW TAGS'))
print(session.execute('SHOW EDGES'))          

In [ ]:
%%time
i = 0
for row in df.itertuples(index = False):
    session.execute('INSERT VERTEX IF NOT EXISTS `{}`(name) VALUES "{}":("{}");'
                    'INSERT VERTEX IF NOT EXISTS `{}`(name) VALUES "{}":("{}");'
                    'INSERT EDGE IF NOT EXISTS `{}`(id) VALUES "{}"->"{}":({})' \
                    .format(df.columns[1], row[1], row[1], df.columns[2], row[2], row[2],\
                     df.columns[0], row[1], row[2], row[0])
                   )

In [ ]:
# Статистика по БД
session.execute('SUBMIT JOB STATS')

In [ ]:
# Проверка совпадения количества вершин
session.execute('SHOW STATS')

In [ ]:
# Типовые запросы к БД
# Поиск всех участников 2 и номера `id события` с помощью GO по vid участника 1 
print(session.execute('GO FROM "Галчевская Карина Владимировна" OVER `id события` \
                      YIELD `id события`.id AS `id`, id($$) AS `Участник события 2`'))

# Поиск всех участников 1 и номера `id события` с помощью GO по vid участника 2 по ребру
# `id события` в обратном направлении 
print(session.execute('GO FROM "Жилейкин Виктор Павлович" OVER `id события` REVERSELY \
                      YIELD `id события`.id AS `id`, id($$) AS `Участник события 1`'))

In [ ]:
# Избыточно
# Поиск номера `id события` с помощью GO и FETCH по vid участника 1
print(session.execute('GO FROM "Офицеров Олег Романович" OVER `id события` \
                      YIELD src(edge) AS s, dst(edge) AS d | \
                      FETCH PROP ON `id события` $-.s -> $-.d \
                      YIELD properties(edge).id AS `id`, dst(edge) AS `Участник события 2`'))

# Поиск номера `id события` с помощью GO и FETCH по vid участника 2 в обратном направлении
print(session.execute('GO FROM "Чемодуров Дамир Русланович" OVER `id события` REVERSELY \
                      YIELD src(edge) AS s, dst(edge) AS d | \
                      FETCH PROP ON `id события` $-.s -> $-.d \
                      YIELD properties(edge).id AS id, src(edge) AS `Участник события 1`'))

In [ ]:
# Поиск с помощью MATCH
print(session.execute('MATCH (a:`ФИО участника события 1`) \
                      RETURN a.`ФИО участника события 1`.name LIMIT 10'))
print(session.execute('MATCH (a)-[e]->(b) WHERE id(a)=="Офицеров Олег Романович" \
                      RETURN properties(e) AS `id`, id(b) AS `Участник события 2`'))

In [ ]:
# session.execute('CLEAR SPACE data_1503')

In [ ]:
# session.execute('DROP SPACE data_1503')

In [ ]:
# release session
session.release()

In [ ]:
connection_pool.close()